---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [4]:
import pandas as pd
import numpy as np
import matplotlib as mpl
# import matplotlib.pyplot as plt
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

# disable chained assignments
pd.options.mode.chained_assignment = None

def p_(s):
    print(s)

def preprocess_categories(df):
    from sklearn.preprocessing import LabelEncoder
    from sklearn.preprocessing import OneHotEncoder
    
#     ohe = OneHotEncoder()
#     print(ohe.fit_transform(df[['disposition']]))
#     return ohe.categories_

    df.loc[:, 'agency_name'] = df.agency_name.astype('category') #.cat.as_ordered()
#     df = df.join(pd.get_dummies(df.agency_name, prefix='agency')).drop('agency_name', axis=1)

    return df

    df.inspector_name = df.inspector_name.astype('category').cat.as_ordered()
    df = df.join(pd.get_dummies(df.inspector_name, prefix='inspector')).drop('inspector_name', axis=1)

    df.disposition = df.disposition.astype('category').cat.as_ordered()
    df = df.join(pd.get_dummies(df.disposition, prefix='disposition')).drop('disposition', axis=1)
    
    df.graffiti_status = df.graffiti_status.fillna(0.0)
    
#     df['disposition_transformed'] = LabelEncoder().fit_transform(df.disposition)
#     return df.loc[:, ['disposition', 'disposition_transformed']]

    return df

def test_max_depth(X_train, X_test, y_train, y_test):
    max_depths = np.linspace(1, 32, 32, endpoint=True) # np.linspace(1, 32, 32, endpoint=True)
    train_results = []
    test_results = []
    for max_depth in max_depths:
        dt = DecisionTreeClassifier(max_depth=max_depth)
        dt.fit(X_train, y_train)
        
        train_pred = dt.predict(X_train)
        false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
        roc_auc = auc(false_positive_rate, true_positive_rate)
        # Add auc score to previous train results
        train_results.append(roc_auc)
        
        y_pred = dt.predict(X_test)
        false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
        roc_auc = auc(false_positive_rate, true_positive_rate)
        # Add auc score to previous test results
        test_results.append(roc_auc)

    from matplotlib.legend_handler import HandlerLine2D
    line1, = plt.plot(max_depths, train_results, 'b', label='Train AUC')
    line2, = plt.plot(max_depths, test_results, 'r', label='Test AUC')
    plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
    plt.ylabel('AUC score')
    plt.xlabel('Tree depth')
    plt.show()

def test_model(model_name, use_GridSearch, use_ohe=True):
    #global train_csv, train_df, prod_csv, prod_df, X, y, X_train, X_test, y_train, y_test
    global train_csv, prod_csv

    ## Read train and prod data into train_df and prod_df
    
    if True:
        # Don't add index now - it will be added later (index_col='ticket_id')
        train_csv = pd.read_csv('train.csv', dtype={'zip_code':'str', 'non_us_str_code':'str', 'grafitti_status':'str'}, encoding='cp1252') # or windows-1252
        prod_csv = pd.read_csv('test.csv', dtype={'zip_code':'str', 'non_us_str_code':'str', 'grafitti_status':'str'}, encoding='cp1252') # or windows-1252
        #return train_csv.iloc[0:1, [10,11,30]]

    train_df = train_csv
    prod_df = prod_csv
    
    #print('\ntrain_df.shape: ', train_df.shape)
    #print('train_df.columns:'); print(train_df.columns)
    #return train_df.head()

    ## Add lat/lon to train_df and prod_df
    
    addresses_csv = pd.read_csv('addresses.csv') #, index_col='ticket_id')
    latlons_csv = pd.read_csv('latlons.csv')
    addlatlons_df = pd.merge(addresses_csv, latlons_csv, left_on='address', right_on='address')
    
    #return addlatlons_df.head()

    #print('151818:')
    #print('train_df:', train_df.loc[train_df.ticket_id == 151818])
    #return train_df.loc[train_df.index == 151818]

    #print(train_df.shape)
    train_df = pd.merge(train_df, addlatlons_df, left_on='ticket_id', right_on='ticket_id')
    #print(train_df.shape)

    #print('151818:')
    #print('train_df:', train_df.loc[train_df.index == 151818])
    #return train_df.loc[train_df.ticket_id == 151818]

    #print(prod_df.shape)
    prod_df = pd.merge(prod_df, addlatlons_df, left_on='ticket_id', right_on='ticket_id')
    #print(prod_df.shape)

    ## Pre train/test split processing

    ### Keep only 'responsible' records for now
    train_df = train_df.loc[train_df.compliance.notnull()]
    #print(train_df.shape)
    #print(train_df.groupby('disposition').sum().compliance) # compliance.nunique()
    #return train_df.disposition.value_counts() # .groupby('disposition').sum().compliance # compliance.nunique()
    
    ### Remove non USA countries
    train_df = train_df.loc[train_df.country == 'USA']
    #return train_df.loc[train_df.country != 'USA']
    
    ### Convert zip_code
    train_df.zip_code = pd.to_numeric(train_df.zip_code.str[0:5], errors='coerce')
    train_df.zip_code = train_df.zip_code.fillna(0)
    #return train_df.zip_code.isnull().sum()
    
    #return X.loc[X.zip_code.str[:2]=='48']
    
    ### train_df
    train_df.grafitti_status = train_df.grafitti_status.fillna(0.0)
    train_df.lat = train_df.lat.fillna(0.0)
    train_df.lon = train_df.lon.fillna(0.0)
    train_df['mail_city'] = train_df['city'].apply(lambda train_df: 1 if train_df=='DETROIT' else 0)
    train_df = train_df.drop('city', axis=1)
#     train_df['mail_zip'] = train_df['zip_code'].astype(str).apply(lambda train_df: 0.75 if train_df[:2]=='48' else 0)
#     train_df = train_df.drop('zip_code', atrain_dfis=1)
#     train_df['mail_state'] = train_df['state'].apply(lambda train_df: 1 if train_df=='MI' else 0)
#     train_df = train_df.drop('state', atrain_dfis=1)
#     train_df['mail_country'] = train_df['country'].apply(lambda train_df: 1 if train_df=='USA' else 0)
#     train_df = train_df.drop('country', atrain_dfis=1)
#     train_df['mail_zip'] = train_df['zip_code'].apply(lambda train_df: 1 if train_df[:2]=='48' else 0)
#     train_df = train_df.drop('state', atrain_dfis=1)
    
#     print(len(train_df.loc[train_df.mail_zip > 0]))
#     print(len(train_df.loc[train_df.mail_zip == 0]))
#     return train_df.loc[train_df.mail_zip > 0].head()    total_count = len(train_df)

    ### prod_df
    prod_df.grafitti_status = prod_df.grafitti_status.fillna(0.0)
    prod_df.lat = prod_df.lat.fillna(0.0)
    prod_df.lon = prod_df.lon.fillna(0.0)
#     prod_df['mail_city'] = prod_df['city'].apply(lambda x: 1 if x=='DETROIT' else 0)
#     prod_df = prod_df.drop('city', axis=1)
#     prod_df['mail_state'] = prod_df['state'].apply(lambda x: 1 if x=='MI' else 0)
#     prod_df = prod_df.drop('state', axis=1)
#     prod_df['mail_country'] = prod_df['country'].apply(lambda x: 1 if x=='USA' else 0)
#     prod_df = prod_df.drop('country', axis=1)

    non_compliant = train_df[train_df.compliance == 0].compliance.count()
    compliant = train_df.compliance.sum()
    total_count = len(train_df)
    print(f'compliant: {int(compliant)}, non-compliant: {non_compliant}, total: {total_count}, % compliant: {(compliant / total_count) * 100}')
    
    ## Set index on train_df and prod_df
    
#     train_df = train_df.set_index('ticket_id')
#     prod_df = prod_df.set_index('ticket_id')
    #return train_df.loc[train_df.index == 151818]

    ## Select columns for X

    #X_cols = ['agency_name', 'inspector_name', 'disposition', 'fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost', 'judgment_amount', 'graffiti_status']
    #X_cols = ['agency_name', 'inspector_name', 'disposition', 'fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost', 'judgment_amount', 'graffiti_status', 'lat', 'lon']
    #X_cols = ['agency_name', 'inspector_name', 'disposition', 'fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost', 'judgment_amount', 'graffiti_status', 'lat', 'lon', 'city', 'zip_code'] #, 'state', 'country'] #, 'zip_code'] #, 'state']
    #X_cols = ['agency_name', 'inspector_name', 'disposition', 'judgment_amount', 'grafitti_status', 'lat', 'lon', 'city', 'state', 'zip_code'] #, 'city', 'zip_code'] #, 'state', 'country'] #, 'zip_code'] #, 'state']
    #X_cols = ['agency_name', 'inspector_name', 'disposition', 'fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost', 'judgment_amount', 'grafitti_status', 'lat', 'lon', 'city', 'state', 'zip_code']
    #X_cols = ['mail_city', 'zip_code', 'fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost', 'judgment_amount', 'grafitti_status', 'lat', 'lon']
    #X_cols = ['fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost', 'judgment_amount', 'grafitti_status', 'lat', 'lon']
    X_cols = ['agency_name', 'inspector_name', 'disposition', 'state', 'mail_city', 'zip_code', 'fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost', 'judgment_amount', 'lat', 'lon']
    #X_cols = ['judgment_amount', 'lat', 'lon']
    
    #print(train_df.isnull().sum())
    
    #X = train_df.iloc[:, np.r_[0:25,31]]
    X = train_df[X_cols] # = train_df.loc[:, X_cols]
    X_prod = prod_df.loc[:, X_cols]
    #y = train_df.loc[:, ['compliance']].fillna(0.0)
    y = train_df[['compliance']] # = train_df.loc[:, ['compliance']]

    #return X.isnull().sum()

    ## Train/test split
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

    ## Post split processing
    
    ### Convert strings to categories
    
    #print(f'Cities frequencies: {X.city.value_counts()}')
    #return
    
    if True:
        if not use_ohe: # model_name in ['DecisionTreeClassifier', 'RandomForestClassifier', 'GradientBoostingClassifier']:
            # Use label encoding
            print('Label encoding')
            
            def process_labels(df, df_name):
                df['state_transformed'] = LabelEncoder().fit_transform(df.disposition)
                df = df.drop('state', axis=1)
                df['agency_transformed'] = LabelEncoder().fit_transform(df.disposition)
                df = df.drop('agency_name', axis=1)
                df['inspector_transformed'] = LabelEncoder().fit_transform(df.disposition)
                df = df.drop('inspector_name', axis=1)
                df['disposition_transformed'] = LabelEncoder().fit_transform(df.disposition)
                df = df.drop('disposition', axis=1)
                print(f'{df_name}.shape: {df.shape}')
                return df

            X = process_labels(X, 'X')
            X_train = process_labels(X_train, 'X_train')
            X_test = process_labels(X_test, 'X_test')
            X_prod = process_labels(X_prod, 'X_prod')

            #return df.loc[:, ['state', 'state_transformed', 'agency_name 'disposition', 'disposition_transformed']]
            #return X_train
            
        else:
            # Use one hot encoding
            print('One Hot Encoding')
            
            for col in ['state', 'inspector_name', 'agency_name', 'disposition']: # ['state', 'city', 'agency_name', 'inspector_name', 'disposition']:
                X.loc[:, col] = X[col].astype('category').cat.as_ordered()
                X_train.loc[:, col] = pd.Categorical(X_train[col], categories = X[col].cat.categories, ordered=True) #X_train[col].astype('category').cat.as_ordered()
                X_test.loc[:, col] = pd.Categorical(X_test[col], categories = X[col].cat.categories, ordered=True)
                X_prod.loc[:, col] = pd.Categorical(X_prod[col], categories = X[col].cat.categories, ordered=True)

            def process_dummies(df, df_name):
    #             for col in df.columns:
    #                 if df[col].dtype == np.object:
    #                     print(df[col].dtype)
    #                     if df_name == 'X':
    #                         df.loc[:, col] = df[col].astype('category').cat.as_ordered()
    #                     else:
    #                         df.loc[:, col] = pd.Categorical(df[col], categories = X[col].cat.categories, ordered=True)
    #                     col_short_name = col.replace('_name', '')
    #                     df = df.join(pd.get_dummies(df[col], prefix=col_short_name)).drop(col, axis=1)
    #                     print(f'Column: {col}, {df_name}.shape: {df.shape}')
                df = pd.concat([df, pd.get_dummies(df.state, prefix='state')], axis=1).drop('state', axis=1)
    #             df = pd.concat([df, pd.get_dummies(df.city, prefix='city')], axis=1).drop('city', axis=1)
                df = pd.concat([df, pd.get_dummies(df.agency_name, prefix='agency')], axis=1).drop('agency_name', axis=1)
                df = pd.concat([df, pd.get_dummies(df.inspector_name, prefix='inspector')], axis=1).drop('inspector_name', axis=1)
                df = pd.concat([df, pd.get_dummies(df.disposition, prefix='disposition')], axis=1).drop('disposition', axis=1)
                print(f'{df_name}.shape: {df.shape}')
                return df

            X = process_dummies(X, 'X')
            X_train = process_dummies(X_train, 'X_train')
            X_test = process_dummies(X_test, 'X_test')
            X_prod = process_dummies(X_prod, 'X_prod')

        #print('X_train.shape: ', X_train.shape)
        #print('X_train.columns:'); print(X_train.columns)
        #print('len(y_train): ', len(y_train))
        #return X_train.isnull().sum()

    ## Scale feature weights and reduce dimensions (PCA)
    
    if model_name in ['SVC']:
        print('Scale and reduce dimensions')
        
        scaler = MinMaxScaler().fit(X_train)
        #scaler = StandardScaler().fit(X_train)
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        pca = PCA(n_components = 2).fit(X_train_scaled)
        X_train_pca = pca.transform(X_train_scaled)
        X_test_pca = pca.transform(X_test_scaled)
    else:
        X_train_pca = X_train
        X_test_pca = X_test

    ## Train classifier
    
#     rows, cols = y_train.shape
#     print(rows)
#     print(len(y_train))
#     return type(y_train) # y_train.shape[0]

    if model_name == 'SVC':
        #grid_values = {'kernel': ['linear','poly','rbf'], 'C': [1, 100], 'gamma': [1, 100]}
        #grid_values = {'kernel': ['rbf'], 'C': [1, 100], 'gamma': [1, 100]}
        #model = SVC(random_state=0).fit(X_train_pca, y_train.values.ravel())
        grid_values = {'kernel': ['linear','rbf']} #, 'C': [100], 'gamma': [100]}
        model = SVC(random_state=0, C=100, gamma=100).fit(X_train_pca, y_train.values.ravel())
        
    elif model_name == 'KNeighborsClassifier':
        grid_values = {'n_neighbors': [1, 5, 10]}
        model = KNeighborsClassifier(random_state=0).fit(X_train, y_train.compliance)

    elif model_name == 'LogisticRegression':
        #grid_values = {'C': [12, 15, 20], 'penalty': ['l1', 'l2']}
        #grid_values = {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}
        #grid_values = {'C': [0.01, 0.1, 1, 10, 100], 'solver': ['lbfgs', 'liblinear', 'sag']}
        grid_values = {'C': [0.06, 0.07, 0.08], 'penalty': ['l1', 'l2']}
        # Best params: {'C': 0.07, 'penalty': 'l1'}, AUC score: 0.749100585876
        model = LogisticRegression(random_state=0).fit(X_train, y_train.values.reshape(-1,))

    elif model_name == 'DecisionTreeClassifier':
        #grid_values = {'criterion': ['gini','entropy'], 'max_depth': range(1, 100, 10)}
        #grid_values = {'criterion': ['gini','entropy'], 'max_depth': range(5, 14, 1), 'min_samples_split': np.arange(0.1, 1.0, 0.2)}
        #grid_values = {'criterion': ['gini','entropy'], 'max_depth': range(5, 14, 1), 'min_samples_split': range(50, 110, 10)}
        #grid_values = {'criterion': ['gini','entropy'], 'max_depth': range(5, 14, 1), 'min_samples_split': [0.05,0.25,0.45,0.65,0.85,1.0]}
        #grid_values = {'criterion': ['gini','entropy'], 'max_depth': range(10, 21, 2), 'min_samples_split': [0.01,0.03,0.05]}
        #grid_values = {'criterion': ['gini','entropy'], 'max_depth': range(18, 50, 10), 'min_samples_split': range(2, 60, 10)}
        grid_values = {'criterion': ['gini','entropy'], 'max_depth': range(18, 25, 2), 'min_samples_split': [0.04,0.05,0.06]}
        model = DecisionTreeClassifier(random_state=0).fit(X_train, y_train.values.reshape(-1,))
    
    elif model_name == 'RandomForestClassifier':
        #test_max_depth(X_train, X_test, y_train, y_test)
        #return
        grid_values = {'criterion': ['gini','entropy'], 'max_depth': range(18, 25, 2), 'min_samples_split': [0.04,0.05,0.06]}
        #grid_values = {'criterion': ['entropy'], 'max_depth': [100], 'min_samples_split': [0.04], 'n_estimators': [140, 150, 160]}
        model = RandomForestClassifier(random_state=0).fit(X_train, y_train.values.ravel())
        
    elif model_name == 'GradientBoostingClassifier':
        #grid_values = {'criterion': ['friedman_mse', 'mse', 'mae'], 'n_estimators': [50, 100]} #, 'max_depth': range(18, 25, 2), 'min_samples_split': [0.04,0.05,0.06]}
        #grid_values = {'criterion': ['friedman_mse', 'mse', 'mae'], 'n_estimators': [10], 'learning_rate': [0.1, 0.5]} #, 'max_depth': range(18, 25, 2), 'min_samples_split': [0.04,0.05,0.06]}
        grid_values = {'criterion': ['friedman_mse'], 'n_estimators': [50, 60], 'learning_rate': [0.5, 0.8]} #, 'max_depth': range(18, 25, 2), 'min_samples_split': [0.04,0.05,0.06]}
        model = GradientBoostingClassifier(random_state=0).fit(X_train, y_train.values.ravel())

    if use_GridSearch:
        print('model before GridSearchCV', model)
        print('grid_values', grid_values)
        grid = GridSearchCV(model, param_grid = grid_values, scoring = 'roc_auc')
        grid.fit(X_train_pca, y_train.values.ravel()) #.values.reshape(-1,1)) #.compliance) #values.ravel())
        model = grid.best_estimator_

    if hasattr(model, 'decision_function'):
        y_scores = model.decision_function(X_test_pca)
    else:
        y_scores = model.predict_proba(X_test_pca)[:,1]

    #if hasattr(model, 'tree_'): print('Best max_depth:', model.tree_.max_depth)
    if use_GridSearch:
        print('Best params:', grid.best_params_)
        print('Best score:', grid.best_score_)

    print('AUC score:', roc_auc_score(y_test, y_scores))
    print('y_scores:', y_scores)
    
    return model
    
    print('X_train score:', model.score(X_train, y_train))
    print('X_test score:', model.score(X_test, y_test))
    
    #print('\nX_test.shape: ', X_test.shape)
    #print('X_test.columns:'); print(X_test.columns)
    #print('len(y_test): ', len(y_test))

    y_hat = model.predict(X_test)
    
    #cross_val = cross_val_score(model, X, y, cv=5, scoring = 'roc_auc')
    #print('Cross val score:', cross_val)
    
    auc_score = roc_auc_score(y_test, y_hat)
    print('AUC score:', auc_score)
    
#     false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_hat)
#     roc_auc = auc(false_positive_rate, true_positive_rate)
#     print('ROC AUC score:', roc_auc)

    print('y_hat.head(10):', y_hat) #[:10,])
    
    y_test['y_hat'] = y_hat # model.predict(X_test)
    #proba = model.predict_proba(X_test)[:,1]
    proba = model.decision_function(X_test) # Use with SVM

    y_hat_compliant = y_test.y_hat.sum()
    y_test_compliant = y_test.compliance.sum()
#     y_test_compliant = y_test.loc[y_test.compliance == 1].compliance.count()
    
    print(f'compliant (predicted): {int(y_hat_compliant)}, compliant (actual): {int(y_test_compliant)}, % compliant: {(y_hat_compliant / y_test_compliant) * 100}')
    
#     df = y_test.loc[y_test.compliance == 1]
#     df = df.append(df.sum(numeric_only=True), ignore_index=True)
    
#     print(proba)

    return y_test.loc[y_test.compliance != y_test.y_hat]

#     df = pd.merge(X_test, y_test, left_index=True, right_index=True)
#     return df.loc[df.compliance != df.y_hat]

# test_model('SVC', use_GridSearch=True)
test_model('GradientBoostingClassifier', use_GridSearch=True, use_ohe=False)

compliant: 11595, non-compliant: 148274, total: 159869, % compliant: 7.252813240840938
Label encoding
X.shape: (159869, 15)
X_train.shape: (119901, 15)
X_test.shape: (39968, 15)
X_prod.shape: (61001, 15)
model before GridSearchCV GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)
grid_values {'criterion': ['friedman_mse'], 'n_estimators': [50, 60], 'learning_rate': [0.5, 0.8]

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best params: {'criterion': 'friedman_mse', 'learning_rate': 0.5, 'n_estimators': 50}
Best score: 0.8087291168565779
AUC score: 0.8097222857897693
y_scores: [-4.43566251 -3.11612798 -2.71588659 ... -3.49387409 -3.30162237
 -2.91182094]


GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.5, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='auto',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [1]:
#1

In [8]:
import pandas as pd
import numpy as np
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# from pandas.api.types import is_string_dtype
# from pandas.api.types import is_numeric_dtype
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
# from sklearn.svm import SVC
# from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import GridSearchCV

# disable chained assignments
pd.options.mode.chained_assignment = None

def blight_model():
    global train_csv, prod_csv

    ## Read train and prod data into train_df and prod_df
    
    if True:
        # Don't add index now - it will be added later (index_col='ticket_id')
        train_csv = pd.read_csv('train.csv', dtype={'zip_code':'str', 'non_us_str_code':'str', 'grafitti_status':'str'}, encoding='cp1252') # or windows-1252
        prod_csv = pd.read_csv('test.csv', dtype={'zip_code':'str', 'non_us_str_code':'str', 'grafitti_status':'str'}, encoding='cp1252') # or windows-1252
        #return train_csv.iloc[0:1, [10,11,30]]

    train_df = train_csv
    prod_df = prod_csv

    #print('151818:')
    #print('train_df:', train_df.loc[train_df.index == 151818])
    #return train_df.loc[train_df.index == 151818]
    
    #print('\ntrain_df.shape: ', train_df.shape)
    #print('train_df.columns:'); print(train_df.columns)
    #return train_df.head()

    ## Add lat/lon to train_df and prod_df
    
    addresses_csv = pd.read_csv('addresses.csv') #, index_col='ticket_id')
    latlons_csv = pd.read_csv('latlons.csv')
    addlatlons_df = pd.merge(addresses_csv, latlons_csv, left_on='address', right_on='address')
    
    #return addlatlons_df.head()

    #print('151818:')
    #print('train_df:', train_df.loc[train_df.ticket_id == 151818])
    #return train_df.loc[train_df.index == 151818]

    #print(train_df.shape)
    train_df = pd.merge(train_df, addlatlons_df, left_on='ticket_id', right_on='ticket_id')
    #print(train_df.shape)

    #print('151818:')
    #print('train_df:', train_df.loc[train_df.index == 151818])
    #return train_df.loc[train_df.ticket_id == 151818]

    #print(prod_df.shape)
    prod_df = pd.merge(prod_df, addlatlons_df, left_on='ticket_id', right_on='ticket_id')
    #print(prod_df.shape)

    ## Pre train/test split processing

    ### Keep only 'responsible' records for now
    train_df = train_df.loc[train_df.compliance.notnull()]
    #print(train_df.shape)
    #print(train_df.groupby('disposition').sum().compliance) # compliance.nunique()
    #return train_df.disposition.value_counts() # .groupby('disposition').sum().compliance # compliance.nunique()
    
    ### Remove non USA countries
    train_df = train_df.loc[train_df.country == 'USA']
    #return train_df.loc[train_df.country != 'USA']
    
    ### Convert zip_code
    train_df.zip_code = pd.to_numeric(train_df.zip_code.str[0:5], errors='coerce')
    train_df.zip_code = train_df.zip_code.fillna(0)
    #return train_df.zip_code.isnull().sum()
    prod_df.zip_code = pd.to_numeric(prod_df.zip_code.str[0:5], errors='coerce')
    prod_df.zip_code = prod_df.zip_code.fillna(0)
    
    #return X.loc[X.zip_code.str[:2]=='48']
    
    ### train_df
    train_df.grafitti_status = train_df.grafitti_status.fillna(0.0)
    train_df.lat = train_df.lat.fillna(0.0)
    train_df.lon = train_df.lon.fillna(0.0)
    train_df['mail_city'] = train_df['city'].apply(lambda x: 1 if x=='DETROIT' else 0)
    train_df = train_df.drop('city', axis=1)
#     train_df['mail_zip'] = train_df['zip_code'].astype(str).apply(lambda train_df: 0.75 if train_df[:2]=='48' else 0)
#     train_df = train_df.drop('zip_code', atrain_dfis=1)
#     train_df['mail_state'] = train_df['state'].apply(lambda train_df: 1 if train_df=='MI' else 0)
#     train_df = train_df.drop('state', atrain_dfis=1)
#     train_df['mail_country'] = train_df['country'].apply(lambda train_df: 1 if train_df=='USA' else 0)
#     train_df = train_df.drop('country', atrain_dfis=1)
#     train_df['mail_zip'] = train_df['zip_code'].apply(lambda train_df: 1 if train_df[:2]=='48' else 0)
#     train_df = train_df.drop('state', atrain_dfis=1)
    
#     print(len(train_df.loc[train_df.mail_zip > 0]))
#     print(len(train_df.loc[train_df.mail_zip == 0]))
#     return train_df.loc[train_df.mail_zip > 0].head()    total_count = len(train_df)

    ### prod_df
    prod_df.grafitti_status = prod_df.grafitti_status.fillna(0.0)
    prod_df.lat = prod_df.lat.fillna(0.0)
    prod_df.lon = prod_df.lon.fillna(0.0)
    prod_df['mail_city'] = prod_df['city'].apply(lambda x: 1 if x=='DETROIT' else 0)
    prod_df = prod_df.drop('city', axis=1)
#     prod_df['mail_state'] = prod_df['state'].apply(lambda x: 1 if x=='MI' else 0)
#     prod_df = prod_df.drop('state', axis=1)
#     prod_df['mail_country'] = prod_df['country'].apply(lambda x: 1 if x=='USA' else 0)
#     prod_df = prod_df.drop('country', axis=1)

    non_compliant = train_df[train_df.compliance == 0].compliance.count()
    compliant = train_df.compliance.sum()
    total_count = len(train_df)
#     print(f'compliant: {int(compliant)}, non-compliant: {non_compliant}, total: {total_count}, % compliant: {(compliant / total_count) * 100}')
    
    ## Set index on train_df and prod_df
    
    train_df = train_df.set_index('ticket_id')
    prod_df = prod_df.set_index('ticket_id')
    #return train_df.loc[train_df.index == 151818]

    ## Select columns for X

    #X_cols = ['agency_name', 'disposition', 'state', 'mail_city', 'zip_code', 'fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost', 'judgment_amount', 'grafitti_status', 'lat', 'lon']
    X_cols = ['agency_name', 'disposition', 'state', 'mail_city', 'zip_code', 'fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost', 'judgment_amount', 'lat', 'lon']
    #X_cols = ['zip_code', 'fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost', 'judgment_amount', 'lat', 'lon']
    #X_cols = ['fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost', 'judgment_amount', 'lat', 'lon']
    #X_cols = ['mail_city', 'fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost', 'judgment_amount', 'lat', 'lon']
    
    #print(train_df.isnull().sum())
    
    #X = train_df.iloc[:, np.r_[0:25,31]]
    X = train_df.loc[:, X_cols] # train_df[X_cols] # = 
    X_prod = prod_df.loc[:, X_cols] #  # = 
    #y = train_df.loc[:, ['compliance']].fillna(0.0)
    y = train_df[['compliance']] # = train_df.loc[:, ['compliance']]

    #return X.isnull().sum()

    ## Train/test split
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    
    #print('151818:')
    #print('X:', X.loc[X.index == 151818])
    #print('train_df:', train_df.loc[train_df.index == 151818])
    #return y_test
    #print('y_train:', y_train.loc[y_train.index == 151818])
    #return X_test.loc[X_test.index == 151818]
    #return train_df.loc[train_df.ticket_id == 151818]

    #print('X_test.head(10):', X_test.index[:10]) #.iloc[:10,0])
    #print('y_test.head(10):', y_test.head(10))
    #return

    ## Post split processing
    
    ### Convert strings to categories
    
    if True:
        for col in ['state', 'agency_name', 'disposition']: # ['state', 'city', 'agency_name', 'inspector_name', 'disposition']:
            X.loc[:, col] = X[col].astype('category').cat.as_ordered()
            X_train.loc[:, col] = pd.Categorical(X_train[col], categories = X[col].cat.categories, ordered=True) #X_train[col].astype('category').cat.as_ordered()
            X_test.loc[:, col] = pd.Categorical(X_test[col], categories = X[col].cat.categories, ordered=True)
            X_prod.loc[:, col] = pd.Categorical(X_prod[col], categories = X[col].cat.categories, ordered=True)

        def process_dummies(df, df_name):
#             for col in df.columns:
#                 if df[col].dtype == np.object:
#                     print(df[col].dtype)
#                     if df_name == 'X':
#                         df.loc[:, col] = df[col].astype('category').cat.as_ordered()
#                     else:
#                         df.loc[:, col] = pd.Categorical(df[col], categories = X[col].cat.categories, ordered=True)
#                     col_short_name = col.replace('_name', '')
#                     df = df.join(pd.get_dummies(df[col], prefix=col_short_name)).drop(col, axis=1)
#                     print(f'Column: {col}, {df_name}.shape: {df.shape}')
            df = pd.concat([df, pd.get_dummies(df.state, prefix='state')], axis=1).drop('state', axis=1)
#             df = pd.concat([df, pd.get_dummies(df.city, prefix='city')], axis=1).drop('city', axis=1)
            df = pd.concat([df, pd.get_dummies(df.agency_name, prefix='agency')], axis=1).drop('agency_name', axis=1)
#             df = pd.concat([df, pd.get_dummies(df.inspector_name, prefix='inspector')], axis=1).drop('inspector_name', axis=1)
            df = pd.concat([df, pd.get_dummies(df.disposition, prefix='disposition')], axis=1).drop('disposition', axis=1)
#             print(f'{df_name}.shape: {df.shape}')
            return df

        X = process_dummies(X, 'X')
        X_train = process_dummies(X_train, 'X_train')
        X_test = process_dummies(X_test, 'X_test')
        X_prod = process_dummies(X_prod, 'X_prod')

        #print('X_train.shape: ', X_train.shape)
        #print('X_train.columns:'); print(X_train.columns)
        #print('len(y_train): ', len(y_train))
        #return X_train.isnull().sum()

    ## Scale feature weights
    
    #scaler = MinMaxScaler().fit(X_train)
    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    X_prod_scaled = scaler.transform(X_prod)

    ## use PCA to optimize dimensions - 95% of variance
    
    #pca = PCA(n_components = 2).fit(X_train_scaled)
    #X_train_pca = pca.transform(X_train_scaled)
    #X_test_pca = pca.transform(X_test_scaled)

    ## Optimize tests
    
    #test_max_depth(X_train, X_test, y_train, y_test)
    #return

    ## Train classifier
    
#     rows, cols = y_train.shape
#     print(rows)
#     print(len(y_train))
#     return type(y_train) # y_train.shape[0]
    
    #model = DecisionTreeClassifier(criterion='gini', max_depth=24, min_samples_split=0.05).fit(X_train, y_train.values.ravel())
    model = LogisticRegression(C=0.07, penalty='l1').fit(X_train_scaled, y_train.values.ravel())
    #model = RandomForestClassifier(criterion='entropy', max_depth=100, min_samples_split=0.04, n_estimators=150).fit(X_train, y_train.values.ravel())

#     if hasattr(model, 'predict_proba'):
#         y_scores = model.predict_proba(X_test_scaled)[:,1]
#     else:
#         y_scores = model.decision_function(X_test_scaled)

#     print('AUC score:', roc_auc_score(y_test, y_scores))
    
#     return
#     print('y_scores:', y_scores)
    
#     print('X_train score:', model.score(X_train_scaled, y_train))
#     print('X_test score:', model.score(X_test_scaled, y_test))
    
    #print('\nX_test.shape: ', X_test.shape)
    #print('X_test.columns:'); print(X_test.columns)
    #print('len(y_test): ', len(y_test))

    y_prod_scores = model.predict_proba(X_prod_scaled)[:,1]

#     df = pd.DataFrame(y_prod_scores, columns=['compliance'])
#     return df.compliance.value_counts()
    
#     return y_prod_scores[y_prod_scores > 0.5]
    
#     #print(y_prod_scores[:20])
    
    X_prod['compliance'] = y_prod_scores
    c = X_prod['compliance']
    #print(X_prod.head(20))
    
#     print((len(c[c > 0.5]) / len(c)) * 100)
    
    return c #[c > 0.5] # c.head(10) [c.compliance > 0]

blight_model()

ticket_id
284932    0.046069
285362    0.017417
285361    0.053132
285338    0.055119
285346    0.062187
285345    0.055125
285347    0.066027
285342    0.471088
285530    0.006725
284989    0.027676
285344    0.065854
285343    0.020460
285340    0.017783
285341    0.057587
285349    0.044829
285348    0.039655
284991    0.029448
285532    0.033732
285406    0.032509
285001    0.027694
285006    0.014793
285405    0.014611
285337    0.012144
285496    0.064463
285497    0.053804
285378    0.014524
285589    0.031776
285585    0.047241
285501    0.052969
285581    0.017352
            ...   
376367    0.017296
376366    0.032308
376362    0.276243
376363    0.289224
376365    0.017296
376364    0.032308
376228    0.037354
376265    0.037555
376286    0.228245
376320    0.037312
376314    0.031877
376327    0.230917
376385    0.225066
376435    0.503194
376370    0.398595
376434    0.064424
376459    0.061783
376478    0.000124
376473    0.036962
376484    0.035271
376482    0.029138
37